**38. Si decimos que la ubicación de un usuario es el promedio de la latitud y longitud de
los contenidos geolocalizados para los cuales editó la última versión (ignorar
usuarios que no editaron contenido geolocalizado). ¿Cuáles son los dos usuarios
más cercanos?**

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
#!apt install default-jre
#!apt install default-jdk
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 281.4 MB 23 kB/s 
     |████████████████████████████████| 198 kB 50.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=65ba05ae07c9f06b9920c56e36fc625ba9e931265b35a652f6145f16c93ef9c3
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
0% [1 InRelease gpgv 1,581 B] [4 InRelease 15.6 kB/88.7 kB 18%] [Waiting for he

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

Para este ejercicio voy a utilizar el csv contents y geo_tags

In [ ]:
downloaded = drive.CreateFile({'id':'1SnCVFJvTsaEPrmh-PpVGH6-toBVsD6NL'})
downloaded.GetContentFile('contents.csv')

downloaded2 = drive.CreateFile({'id':'1qLu79VkVJxWp9i4Pnq9cxSLjW6Nipxp9'})
downloaded2.GetContentFile('geo_tags.csv')

In [ ]:
!pip install pyarrow
texts = pd.read_csv('contents.csv')
texts.to_parquet('contents.parquet')
del texts

sqlContext = SQLContext(sc)
df = sqlContext.read.parquet('contents.parquet')
rdd1 = df.rdd

In [ ]:
texts = pd.read_csv('geo_tags.csv')
texts.to_parquet('geo_tags.parquet')
del texts

sqlContext = SQLContext(sc)
df2 = sqlContext.read.parquet('geo_tags.parquet')
rdd2 = df2.rdd

In [ ]:
contents = rdd1.map(lambda x: (x.id, x.revisor_username if x.revisor_username != None else x.revisor_ip))
contents.take(3)

In [ ]:
geo = rdd2.map(lambda x: (x.gt_page_id, (x.gt_lat, x.gt_lon)))
geo.take(3)

In [ ]:
join = contents.join(geo)
join.take(5)

In [ ]:
join.count()

Mapeo para que me quede de la forma: USUARIO - LATITUD(contenido) - LONGITUD(contenido)

In [ ]:
join.map(lambda x: (x[1][0], (1, x[1][1][0], x[1][1][1]))).take(5)

Reduzco por los usuarios, contando sus ocurrencias y sumando las latitudes y longitudes para posteriormente calcular los promedios

In [ ]:
join.map(lambda x: (x[1][0], (1, x[1][1][0], x[1][1][1]))).reduceByKey(lambda a,b: (a[0]+b[0], a[1]+b[1], a[2]+b[2])).take(5)

Finalmente tengo a los usuario con sus posiciones:

In [ ]:
join.map(lambda x: (x[1][0], (1, x[1][1][0], x[1][1][1])))\
  .reduceByKey(lambda a,b: (a[0]+b[0], a[1]+b[1], a[2]+b[2]))\
  .map(lambda x: (x[0], (x[1][1]/x[1][0],x[1][2]/x[1][0]))).take(5)

In [ ]:
usuarios = join.map(lambda x: (x[1][0], (1, x[1][1][0], x[1][1][1])))\
                .reduceByKey(lambda a,b: (a[0]+b[0], a[1]+b[1], a[2]+b[2]))\
                .map(lambda x: (x[0], (x[1][1]/x[1][0],x[1][2]/x[1][0]))).cache()

Voy a utilizar un BallTree para poder buscar el vecino más cercano de cada usuario.

In [ ]:
from sklearn.neighbors import BallTree
import numpy as np

lats = usuarios.map(lambda x: x[1][0]).collect()
lons = usuarios.map(lambda x: x[1][1]).collect()

bt = BallTree(np.deg2rad(usuarios.map(lambda x: (x[1][0],x[1][1])).collect()), metric='haversine')
distances, indices = bt.query(np.deg2rad(np.c_[lats, lons]), k=2)

El array distances tendrá las distancias de los 2 vecinos más cercanos, donde siempre el primer valor será 0 (distancia a el mismo) y el segundo tendrá un valor distinto de cero.

El array indices tendrá también 2 valores, el primero será el mismo y el 2do será el indice del vecino más cercano (distinto de él)

Aunque en este set de datos hay casos particulares, los cuales son usuarios en exactamente la misma ubicación (esto hace que a veces se calcule primero la distancia al otro y luego a si mismo (los datos estarían cruzados))

Paso las distancias a KM multiplicando por 6371 que es el radio de la tierra en km.

In [ ]:
radio = 6371
for i in range(0, len(distances)):
  distances[i][1] = distances[i][1] * radio

print(distances)

In [ ]:
print(indices)

Ahora voy a identificar cada usuario con su indice correspondiente.

El usuario 0 es el primer usuario que aparece en el RDD **usuarios**

In [ ]:
lista_usuarios = usuarios.map(lambda x: x[0]).collect()
aux = np.c_[indices, lista_usuarios].tolist()

for elem in aux:
  for i in range(1, len(aux)):
    if elem[1] == aux[i][0]:
        elem.append(aux[i][2])
        break

Entonces, para el usuario 'Lojwe' el más cercano es el usuario 'Rodm23 mantenimiento' y viceversa:

In [ ]:
aux[0]

Están a una distancia de 85,76 km:

In [ ]:
distances[0]

In [ ]:
distances[332]

Junto todo:

In [ ]:
for i in range(0, len(aux)):
  aux[i].append((distances[i][0], distances[i][1]))

aux[0]

Convierto a RDD y mapeo con los datos que necesito:

USUARIO - VECINO + CERCANO - DISTANCIA(KM)

In [ ]:
rdd_vecinos = sc.parallelize(aux).map(lambda x: (x[2], (x[3], x[4][1])))

In [ ]:
rdd_vecinos.take(5)


### **ACLARACIÓN**

No pude trabajar con este RDD que arme y no estaría entendiendo el porqué.
Quise reducir para encontrar el minimo y no puedo.
Quiero filtrar por un numero y tampoco puedo.

Ya casi es la hora de entrega, así que por lo tanto lo dejo así. Tuve poco tiempo para realizar el tp y estoy por entregar a ultimo momento si no le hubiera consultado a mi corrector sobre esto. 

Lo que me hubiera gustado hacer para terminar:

1) Aplicar un reduce para encontrar la distancia minima.

2) Filtrar por esa distancia y obtener los usuarios mas cercanos

Este ejercicio me toco en el tp de Pandas, resulta que hay casos particulares con distancia 0 ya que están en la misma posición.

Lo que hubiera hecho, sería comprobar lo anterior, y luego volver a hacer lo mismo pero con el segundo minimo (distinto de cero)

Dejo los problemas que tuve:

In [ ]:
type(rdd_vecinos)

In [ ]:
rdd_vecinos.reduce(lambda a,b: a[1][1] if a[1][1] < b[1][1] else b[1][1])

In [ ]:
rdd_vecinos.map(lambda x: x[1][1]).reduce(lambda a,b: a if a < b else b)

In [ ]:
rdd_vecinos.filter(lambda x: x[1][1] == 0).collect()